# NUSA Demo of Fama French Factor Model

In [2]:
import pandas as pd
import numpy as np
# use any other libraries you may need

Read the contents of **cleaned_factset_data.csv**  into a Pandas Dataframe called **df** and drop any rows with NaN values. Note that the **CAP** column values are Strings with commas to denote thousands, so convert all the values in the column to Floats.

In [3]:
# reading data
df = pd.read_csv('cleaned_factset_data.csv')
df.head()

,Ticker,Company Name,monthly_return,capm_beta,book_price,CAP,GPM
0,DDD,3D Systems Corporation,-6.02,1.555648,0.436308,1523.9963,48.936516
1,MMM,3M Company,4.50,1.079156,0.074971,125018.1300,49.739280
2,EGHT,"8x8, Inc.",-1.81,0.366954,0.236263,1241.3590,75.486600
3,AOS,A. O. Smith Corporation,-1.52,1.536893,0.147579,11333.7530,41.665737
4,SHLM,"A. Schulman, Inc.",9.18,1.600787,0.033661,1006.2639,16.560259


To reduce the impact of outliers caused by the few number of large cap companies, add a new column to **df** called **log_mktcap** and populate it with the log of each value in **CAP**. 

In [4]:
#finding log value of CAP column
df['log_mktcap'] = np.log(df['CAP'])

In [5]:
df

,Ticker,Company Name,monthly_return,capm_beta,book_price,CAP,GPM,log_mktcap
0,DDD,3D Systems Corporation,-6.02,1.555648,0.436308,1523.99630,48.936516,7.329091
1,MMM,3M Company,4.50,1.079156,0.074971,125018.13000,49.739280,11.736214
2,EGHT,"8x8, Inc.",-1.81,0.366954,0.236263,1241.35900,75.486600,7.123962
3,AOS,A. O. Smith Corporation,-1.52,1.536893,0.147579,11333.75300,41.665737,9.335541
4,SHLM,"A. Schulman, Inc.",9.18,1.600787,0.033661,1006.26390,16.560259,6.914000
...,...,...,...,...,...,...,...,...
1501,ZBRA,Zebra Technologies Corporation Class A,3.12,1.604373,0.129115,5766.58000,39.653730,8.659834
1502,ZBH,"Zimmer Biomet Holdings, Inc.",0.98,1.182440,0.396316,23663.88900,60.496624,10.071705
1503,ZION,Zions Bancorporation,NaN,1.470795,0.733912,9422.41200,NaN,9.150846
1504,ZTS,"Zoetis, Inc. Class A",-1.51,1.016201,0.047275,31104.16800,64.566284,10.345097


Then calculate the z-score of each of the numeric columns and put the results into new columns with **'zscore_'** prepended to each original column name. 


The z-score formula is:

|      $Z = \frac{x - \mu}{\sigma}$

Where $\mu$ is the column mean, $\sigma$ is the column standard deviation, and $x$ is the observed value.


In [6]:
#defining zscore function
def z_score(col):
    df['zscore_' + col] = (df[col] - df[col].mean()) / df[col].std()
    return df

#applying zscore through each column
for column in df.columns[2:]:
    z_score(column)

df.head()

,Ticker,Company Name,monthly_return,capm_beta,book_price,CAP,GPM,log_mktcap,zscore_monthly_return,zscore_capm_beta,zscore_book_price,zscore_CAP,zscore_GPM,zscore_log_mktcap
0,DDD,3D Systems Corporation,-6.02,1.555648,0.436308,1523.9963,48.936516,7.329091,-0.927340,0.618140,-0.046824,-0.302299,0.558255,-0.691310
1,MMM,3M Company,4.50,1.079156,0.074971,125018.1300,49.739280,11.736214,0.938459,-0.066353,-0.747973,2.053508,0.594841,2.182752
2,EGHT,"8x8, Inc.",-1.81,0.366954,0.236263,1241.3590,75.486600,7.123962,-0.180666,-1.089453,-0.434997,-0.307691,1.768273,-0.825083
3,AOS,A. O. Smith Corporation,-1.52,1.536893,0.147579,11333.7530,41.665737,9.335541,-0.129232,0.591199,-0.607082,-0.115166,0.226890,0.617177
4,SHLM,"A. Schulman, Inc.",9.18,1.600787,0.033661,1006.2639,16.560259,6.914000,1.768491,0.682984,-0.828131,-0.312176,-0.917290,-0.962008


Winsorize the data in the **'zscore'** columns at the 1st and 99th percentiles. 
(Censor the outliers, set any values less than the 1st percentile to the value of the 1st percentile and any values greater than the 99th percentile to the value at the 99th percentile).

In [7]:
from scipy.stats import zscore
from scipy.stats.mstats import winsorize

In [8]:
#setting the limits for the winsorization
lowest_percentile = 1
highest_percentile = 99

for col in df.columns:
    # winsorizing through each column of the df
    df[col] = winsorize(df[col], limits = (lowest_percentile/100, highest_percentile/100))

df.head()


c:\Users\harsh\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "


,Ticker,Company Name,monthly_return,capm_beta,book_price,CAP,GPM,log_mktcap,zscore_monthly_return,zscore_capm_beta,zscore_book_price,zscore_CAP,zscore_GPM,zscore_log_mktcap
0,ABM,"ALLETE, Inc.",-16.44,0.039155,-0.145624,239.49306,-10.021059,5.478524,-2.775403,-1.560345,-1.176021,-0.326803,-2.128731,-1.898139
1,ABM,"ALLETE, Inc.",-16.44,0.039155,-0.145624,239.49306,-10.021059,5.478524,-2.775403,-1.560345,-1.176021,-0.326803,-2.128731,-1.898139
2,ABM,"ALLETE, Inc.",-16.44,0.039155,-0.145624,239.49306,-10.021059,5.478524,-2.775403,-1.560345,-1.176021,-0.326803,-2.128731,-1.898139
3,ABM,"ALLETE, Inc.",-16.44,0.039155,-0.145624,239.49306,-10.021059,5.478524,-2.775403,-1.560345,-1.176021,-0.326803,-2.128731,-1.898139
4,ABM,"ALLETE, Inc.",-16.44,0.039155,-0.145624,239.49306,-10.021059,5.478524,-2.775403,-1.560345,-1.176021,-0.326803,-2.128731,-1.898139


Run a **weighted least squares regression** using the standardized, winsorized data as explanatory variables and the monthly returns as the dependent.

In [9]:
import statsmodels.api as sm 
import matplotlib.pyplot as plt

# Define the independent variables (explanatory variables)
X = df[['zscore_capm_beta', 'zscore_book_price', 'zscore_CAP', 'zscore_GPM', 'zscore_log_mktcap']]

# Add a constant term for the intercept
X = sm.add_constant(X)

# Define the dependent variable
y = df['monthly_return']

# Perform WLS regression
model = sm.WLS(y, X, weights=1)  # Using weights=1 for uniform weighting, adjust as needed
results = model.fit()

# Display regression summary
print(results.summary())

                            WLS Regression Results                            
Dep. Variable:         monthly_return   R-squared:                        -inf
Model:                            WLS   Adj. R-squared:                   -inf
Method:                 Least Squares   F-statistic:                    -1504.
Date:                Sat, 23 Sep 2023   Prob (F-statistic):               1.00
Time:                        16:50:25   Log-Likelihood:                -4603.4
No. Observations:                1506   AIC:                             9211.
Df Residuals:                    1504   BIC:                             9222.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
zscore_capm_beta    -24.1328    2.3e+1

c:\Users\harsh\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:125: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew = stats.skew(resids, axis=axis)
c:\Users\harsh\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:126: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis = 3 + stats.kurtosis(resids, axis=axis)
c:\Users\harsh\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:1667: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  b2 = skew(a, axis)
c:\Users\harsh\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:1771: RuntimeWarning: Precision loss occurred in moment calculation due to c

**Write a sentence or two interpreting the results of the regression, what do the coefficients mean and are they statistically significant?**

The coefficients indicate the strength and direction of the relationshup between each independent variable and the dependent variable (monthly returns in this case)